In [ ]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
from transformers import pipeline

model_id = "llava-hf/llava-1.5-7b-hf" # Change model here
pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

In [ ]:
!pip install datasets
!pip install evaluate

In [ ]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

dataset = load_dataset("AI4Math/MathVista")

In [ ]:
data_1000_llava = pd.DataFrame()
data_1000_llava['category']=""
categories=[]
ids =[]
prompt_cat = "There are 4 types of possible tasks that the following mathematical problem could be classified into. These are Figure Question Answering (FQA), Textbook Question answering (TQA), Math Word Problems (MWP), Geometry Problem solving (GPS). Provide the task of the problem in the following return prompt and replace the ### by the right task acronym : The task : ###."
for id in tqdm(dataset['testmini']['pid']):
  ids.append(id)
  image_url = dataset["testmini"][int(id)]["decoded_image"]
  prompt = "<image>\n" + dataset["testmini"][int(id)]["query"] +prompt_cat
  outputs = pipe(image_url, prompt=prompt, generate_kwargs={"max_new_tokens": 500})
  outputs = outputs[0]["generated_text"]
  idx = outputs.find("Type of Task:")
  categories.append(outputs[idx+14:])
  data_1000_llava['category'][id] = outputs[idx+14:]

In [ ]:
df.to_csv("category_llava_1000.csv",index=False)

In [ ]:
tasks = {'figure question answering': 'FQA', 'textbook question answering': 'TQA', 'math word problems': 'MWP', 'geometry problem solving': 'GPS'}
tasks_r = {'FQA':'figure question answering', 'TQA' : 'textbook question answering', 'MWP':'math word problems', 'GPS': 'geometry problem solving'}

In [ ]:
count = 0
i = 0
for tsk in categories:
  tsk = tsk[:3]
  if tasks_r[tsk] == dataset['testmini']['metadata'][i]['task']:
    count += 1
  i += 1
print(count)
print(count/i)

In [ ]:
# Get accuracy scores for problem category recognition
ctr = 0
i = 0
score = 0

while ctr<100 and i<1000:
  if(not (dataset["testmini"][i]["metadata"]["language"] == 'english' and dataset["testmini"][i]["metadata"]["category"] == 'math-targeted-vqa')):
    i += 1
    continue
  if dataset['testmini']['metadata'][i]['task'] == tasks_r[categories[ctr][0:3]]:
    score += 1


  ctr += 1

print(score/100)